In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
import scipy.stats
import copy
import random

In [2]:
class Chromossome:
    def __init__(self, algorithm, **hyperparemeter_range):
        self.hyperparemeter_range = hyperparemeter_range
        self.algorithm = algorithm
        self.mutate()
        
    def fit(self, X, y):
        self.classifier.fit(X, y)
        
    def predict(self, X):
        return self.classifier.predict(X)
    
    def mutate(self):
        param = {}
        for hyperparameter, h_range in self.hyperparemeter_range.items():
            if isinstance(h_range[0], str):
                param[hyperparameter] = random.choice(h_range)
            elif isinstance(h_range[0], float):
                param[hyperparameter] = random.uniform(h_range[0], h_range[1])
            else:
                param[hyperparameter] = random.randint(h_range[0], h_range[1])
        self.classifier = self.algorithm(**param)

In [3]:
class DiversityEnsembleClassifier:
    def __init__(self, algorithms, population_size = 100, max_epochs = 100, random_state=None):
        self.population_size = population_size
        self.max_epochs = max_epochs
        self.population = []
        
        for algorithm, hyperparameters in algorithms.items():
            for i in range(0, population_size//len(algorithms.keys())):
                self.population.append(Chromossome(algorithm, **hyperparameters))
        self.population.append(Chromossome(algorithm, **hyperparameters))
        
    def fit(self, X, y):
        print('Running epoch ', end='')
        for epoch in range(self.max_epochs):
            print(epoch, end='...')
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=epoch)
            predictions = np.empty([2*self.population_size, y_val.shape[0]])
            
            for i in range(0, self.population_size):
                new_chromossome = copy.deepcopy(self.population[i])
                new_chromossome.mutate()
                self.population.append(new_chromossome)
                
            for i in range(2*self.population_size):
                chromossome = self.population[i]
                chromossome.fit(X_train, y_train)
                predictions[i] = np.logical_and(chromossome.predict(X_val), y_val)
                       
            distances = np.zeros(2*self.population_size)
            target_chromossome = np.argmax(predictions.sum(axis=1))
            new_population = [self.population[target_chromossome]]            
            
            for i in range(0, self.population_size-1):
                distances[target_chromossome] = float('-inf')
                distances += np.logical_xor(predictions, predictions[target_chromossome]).sum(axis=1)
                target_chromossome = np.argmax(distances)
                new_population.append(self.population[target_chromossome])
                
            self.population =new_population
                        
    def predict(self, X):
        predictions = np.empty([self.population_size, X.shape[0]])
        for i in range(0, self.population_size):
            predictions[i] = self.population[i].predict(X)
        return scipy.stats.mode(predictions, axis=0).mode
        

In [4]:
wine = datasets.load_wine()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2)
alg = {
            KNeighborsClassifier: {'n_neighbors':[1, 100]},
            SVC: {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], 
                  'degree' : [0, 5]
                  }
      }
dec = DiversityEnsembleClassifier(alg, population_size=100, max_epochs=500)
dec.fit(X_train,  y_train)
print('Acuracy', np.logical_and(dec.predict(X_test), y_test).sum()/y_test.shape[0])


In [ ]:
print([x.classifier for x in dec.population])